### Get Gene Info

In [3]:
import pandas as pd
from Bio import Entrez
import Request_Utilis
from google_sheet import GoogleSheet

Entrez.email = "a.antonakoudis@sartorius.com"

In [4]:
from google_sheet import GoogleSheet

##### ----- Generate datasets from Google Sheet ----- #####

#Credential file
KEY_FILE_PATH = 'credentials.json'

#CHO Network Reconstruction + Recon3D_v3 Google Sheet ID
SPREADSHEET_ID = '1DaAdZlvMYDqb7g31I5dw-ZCZH52Xj_W3FnQMFUzqmiQ'

# Initialize the GoogleSheet object
gsheet_file = GoogleSheet(SPREADSHEET_ID, KEY_FILE_PATH)

# Read data from the Google Sheet
sec_recon_sheet = 'Sheet1'
sec_recon = gsheet_file.read_google_sheet(sec_recon_sheet)
sec_recon

,GENE SYMBOL,ALIAS,ENSEMBL,ENTREZID,GENENAME,Mapped Process,SecM,Regulator,SecP,Membrane,...,Process 8 Ref,Process 8 Genes,Process 9,Process 9 Mean Score,Process 9 Score SD,Process 9 Score Count,Process 9 Raw Scores,Process 9 Curators,Process 9 Ref,Process 9 Genes
0,A3GALT2,"A3GALT2P,IGB3S,IGBS3S,A3GALT2",ENSG00000184389,127550,"alpha 1,3-galactosyltransferase 2","Glycosylation', 'Glycosylation'",0,0,0,0,...,None,None,None,None,None,None,None,None,None,None
1,A4GALT,"A14GALT,A4GALT1,Gb3S,P(k),P1,P1PK,PK,A4GALT",ENSG00000128274,53947,"alpha 1,4-galactosyltransferase (P blood group)","glycolipid, globo series', 'globo series', 'gl...",0,0,0,0,...,None,None,None,None,None,None,None,None,None,None
2,A4GNT,"alpha4GnT,A4GNT",ENSG00000118017,51146,"alpha-1,4-N-acetylglucosaminyltransferase","O-glycan', 'O-glycan, mucin-type', 'mucin-type'",0,0,0,0,...,None,None,None,None,None,None,None,None,None,None
3,ABL1,"ABL,BCR-ABL,CHDSKM,JTK7,bcr/abl,c-ABL,c-ABL1,p...",ENSG00000097007,25,"ABL proto-oncogene 1, non-receptor tyrosine ki...",[],1,1,0,0,...,None,None,None,None,None,None,None,None,None,None
4,ABL2,"ABLL,ARG,ABL2",ENSG00000143322,27,"ABL proto-oncogene 2, non-receptor tyrosine ki...",[],1,1,0,0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,YKT6,YKT6,ENSG00000106636,10652,YKT6 v-SNARE homolog,"General Trafficking', 'General Trafficking', '...",0,0,0,1,...,None,None,None,None,None,None,None,None,None,None
912,YOD1,"DUBA8,OTUD2,PRO0907,YOD1",ENSG00000180667,55432,YOD1 deubiquitinase,"ubl conjugation pathway', 'ERAD', 'ERAD', 'ERA...",2,1,0,0,...,None,None,None,None,None,None,None,None,None,None
913,USP19,,,10869,,,1,1,0,0,...,None,None,None,None,None,None,None,None,None,None
914,DGAT1,,,8694,,,1,1,0,1,...,None,None,None,None,None,None,None,None,None,None


In [5]:
sec_recon_dc = sec_recon.copy()
for i,row in sec_recon_dc.iterrows():
    if pd.isnull(row['ENTREZID']) or row['ENTREZID'] == '':
        human_entrez = Request_Utilis.get_entrez_id(row['GENE SYMBOL'])
        sec_recon_dc.at[i, 'ENTREZID'] = human_entrez

if not sec_recon_dc.equals(sec_recon):
    gsheet_file.update_google_sheet(sec_recon_sheet, sec_recon_dc)
    print("Google Sheet updated.")

In [6]:
for gene_EntrezID in sec_recon['ENTREZID']:

    # Get CHO and Mouse Orthologs
    mouse_ortholog_EntrezID = Request_Utilis.get_gene_ids(gene_EntrezID, '10090')
    cho_ortholog_EntrezID = Request_Utilis.get_gene_ids(gene_EntrezID, '10029')
    
    # print Human Info
    print(gene_EntrezID)
    org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products = Request_Utilis.Gene_Info_from_EntrezID(gene_EntrezID)
    print(org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products)
    
    # print Mouse Info
    org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products = Request_Utilis.Gene_Info_from_EntrezID(mouse_ortholog_EntrezID)
    print(org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products)
    
    # print CHO Info
    org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products = Request_Utilis.Gene_Info_from_EntrezID(cho_ortholog_EntrezID)
    print(org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products)
    print("---------------------------------")


        

127550
Homo sapiens A3GALT2 alpha 1,3-galactosyltransferase 2 ['IGB3S', 'IGBS3S', 'A3GALT2P'] ENSG00000184389 [('NM_001080438', 'NP_001073907', ['U3KPV4']), ('NM_001080438', 'NP_001073907', ['U3KPV4'])]
Mus musculus A3galt2 alpha 1,3-galactosyltransferase 2 ['iGb3', 'Gm433', 'iGb3S'] ENSMUSG00000028794 [('NM_001009819', 'NP_001009819', ['B2KGX5', 'B2KGX6', 'M9MMK2', 'Q3V1N9'])]
Cricetulus griseus A3galt2 alpha 1,3-galactosyltransferase 2 [] ENSCGRG00015008603 [('XM_016974743', 'XP_016830232', ['A0A8C2MUL3', 'A0A8C2QMY7']), ('XM_027398983', 'XP_027254784', ['A0A8C2MUL3', 'A0A8C2QMY7'])]
---------------------------------
53947
Homo sapiens A4GALT alpha 1,4-galactosyltransferase (P1PK blood group) ['P1', 'PK', 'Gb3S', 'P(k)', 'P1PK', 'A14GALT', 'A4GALT1'] ENSG00000128274 [('XM_005261648', 'XP_005261705', ['A0A0S2Z5J1', 'A9XEA7', 'B0L3P6', 'B2R7C4', 'I6S6F0', 'Q540I5', 'Q540I6', 'Q6RJV9', 'Q6RJW0', 'Q6RJW1', 'Q7Z2I1', 'Q9NPC4', 'Q9P1X5']), ('NM_001318038', 'NP_001304967', ['A0A0S2Z5J1', 'A

KeyError: 'Gene-ref_db'

In [ ]:
cho_ortholog_EntrezID

In [ ]:
org, gene_symbol, gene_name, gene_synonyms, gene_ensemble, gene_products = Request_Utilis.Gene_Info_from_EntrezID(cho_ortholog_EntrezID)

In [ ]:
def read_gene_ID(gene_EntrezID):
    gene_fetch_tries = 0
    gene_fetch_found = True
    record = []
    while gene_fetch_tries < 3 and gene_fetch_found:
        try:
            search_results = Entrez.esearch(db="gene", term = gene_EntrezID)            
            record = Entrez.read(search_results)
            gene_fetch_found = False
        except Exception as e:
            time.sleep(3)
            gene_fetch_tries += 1
    if gene_fetch_tries > 2:        
        print(f"EntrezID of {gene_EntrezID} could not be read; An error occurred")

    if int(record["Count"]) > 0:

        # Get the Gene ID of the first result
        gene_id = record["IdList"][0]

        # Fetch the gene information, maximum 3 tries
        fetch_tries = 0
        fetch_found = True
        while fetch_tries < 3 and fetch_found:
            gene_fetch_tries = 0
            gene_fetch_found = True
            gene_info = []
            while gene_fetch_tries < 3 and gene_fetch_found:
                try:
                    gene_info = Entrez.efetch(db="gene", id=gene_id, retmode="xml")    
                    gene_fetch_found = False
                except Exception as e:
                    time.sleep(3)
                    gene_fetch_tries += 1
                    print(f"Gene ID {gene_id}; An error occurred: {e}")
            if gene_fetch_tries > 2:
                print(f"{gene_EntrezID}. Gene ID {gene_id}; An error occurred")

            if gene_info == []:
                KeyError("sda")
            
            try:
                gene_record = Entrez.read(gene_info)
                fetch_found = False
            except Exception as e:
                time.sleep(3)
                fetch_tries += 1                
    return gene_record

In [ ]:
EntrezID = 127550
gene_record = read_gene_ID(int(EntrezID))
org = gene_record[0]['Entrezgene_source']['BioSource']['BioSource_org']['Org-ref']['Org-ref_taxname']

In [ ]:
gene_products = []
org = gene_record[0]['Entrezgene_source']['BioSource']['BioSource_org']['Org-ref']['Org-ref_taxname']
if 'Gene-ref_syn' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
    gene_synonyms = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_syn']
else:
    gene_synonyms = []

# Get the gene name and gene NCBI ID
if 'Gene-ref_locus' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
    gene_symbol = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']                        
else:
    gene_symbol = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_locus-tag']

# Get the Gene description
if 'Gene-ref_desc' in gene_record[0]['Entrezgene_gene']['Gene-ref']:
    gene_name = gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']
else:
    gene_name = gene_record[0]['Entrezgene_locus'][0]['Gene-commentary_products'][0]['Gene-commentary_label']

# Get Ensemble ID
for gene_db_refs in gene_record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_db']:
    if gene_db_refs['Dbtag_db'] == 'Ensembl':
        gene_ensemble = (gene_db_refs['Dbtag_tag']['Object-id']['Object-id_str'])

#print("Organism: ", org, "|Entrez ID: ", EntrezID, "|Gene Symbol: ", gene_symbol, "|Gene Name: ", gene_name, "|Gene Synonyms: ", gene_synonyms, "| Gene Ensemble: ", gene_ensemble)

for assembly_specific_info in gene_record[0]['Entrezgene_locus']:  
    if 'Gene-commentary_heading' in assembly_specific_info:
        for assembly_specific_transcript in assembly_specific_info['Gene-commentary_products']:
            if 'Gene-commentary_products' in assembly_specific_transcript:
                transcript_sequence_ID = assembly_specific_transcript['Gene-commentary_accession']

                for Entrez_Comments in gene_record[0]['Entrezgene_comments']:
                    if 'Gene-commentary_comment' in Entrez_Comments:
                        for comments in Entrez_Comments['Gene-commentary_comment']:
                            if 'Gene-commentary_products' in comments:
                                for product_per_assembly in comments['Gene-commentary_products']:                                
                                    if product_per_assembly['Gene-commentary_heading'] == 'mRNA Sequence':
                                        mRNA = product_per_assembly['Gene-commentary_accession']
                                        if mRNA == transcript_sequence_ID:  
                                            for protein_per_transcript_per_assemlby in product_per_assembly['Gene-commentary_products']:
                                                protein_sequence_ID = protein_per_transcript_per_assemlby['Gene-commentary_accession']
                                                
                                                uniprotID_list = []
                                                for protein_comments in protein_per_transcript_per_assemlby['Gene-commentary_comment']:
                                                    if protein_comments['Gene-commentary_heading'] == 'UniProtKB':
                                                        for uniprotID_protein_per_assembly in protein_comments['Gene-commentary_comment'][0]['Gene-commentary_source']:
                                                            protein_uniprot_id = uniprotID_protein_per_assembly['Other-source_src']['Dbtag']['Dbtag_tag']['Object-id']['Object-id_str']
                                                            uniprotID_list.append(protein_uniprot_id)
                                                gene_products.append(((transcript_sequence_ID, protein_sequence_ID, uniprotID_list)))                                       
                                    else:   
                                        if 'Gene-commentary_products' in product_per_assembly:
                                            for transcript_per_assembly in product_per_assembly['Gene-commentary_products']:      
                                                mRNA = transcript_per_assembly['Gene-commentary_accession']          
                                                if mRNA == transcript_sequence_ID:                                                  
                                                    for protein_per_transcript_per_assemlby in transcript_per_assembly['Gene-commentary_products']:                                                                
                                                        protein_sequence_ID = protein_per_transcript_per_assemlby['Gene-commentary_accession']
                                                        uniprotID_list = []
                                                        if 'Gene-commentary_comment' in protein_per_transcript_per_assemlby:
                                                            for protein_comments in protein_per_transcript_per_assemlby['Gene-commentary_comment']:                                                         
                                                                if protein_comments['Gene-commentary_heading'] == 'UniProtKB':
                                                                    for uniprotID_protein_per_assembly in protein_comments['Gene-commentary_comment'][0]['Gene-commentary_source']:
                                                                        protein_uniprot_id = uniprotID_protein_per_assembly['Other-source_src']['Dbtag']['Dbtag_tag']['Object-id']['Object-id_str']
                                                                        uniprotID_list.append(protein_uniprot_id)
                                                        else:
                                                            uniprotID_list = []
                                                        gene_products.append(((transcript_sequence_ID, protein_sequence_ID, uniprotID_list)))
print(org)
print(EntrezID)
print(gene_symbol)
print(gene_name)
print(gene_synonyms)
print(gene_ensemble)
print(gene_products)

In [ ]:
protein_per_transcript_per_assemlby